In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os
import sys
import glob
import cv2

import pandas as pd
import numpy as np
from natsort import natsorted

import matplotlib
matplotlib.use('TkAgg')
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

%matplotlib notebook

In [3]:
x = np.zeros(10)
y = np.zeros(10)
z = np.arange(10) # remove *100 and the arrow heads will reappear.
dx = np.zeros(10)
dy = np.arange(10)
dz = np.zeros(10)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.quiver(x, y, z, dx, dy, dz)
ax.set_ylim(0,10)

plt.show()

<IPython.core.display.Javascript object>

In [4]:

fig = plt.figure()
ax = fig.gca(projection='3d')

# Make the grid
x, y, z = np.meshgrid(np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.8))

# Make the direction data for the arrows
u = np.sin(np.pi * x) * np.cos(np.pi * y) * np.cos(np.pi * z)
v = -np.cos(np.pi * x) * np.sin(np.pi * y) * np.cos(np.pi * z)
w = (np.sqrt(2.0 / 3.0) * np.cos(np.pi * x) * np.cos(np.pi * y) *
     np.sin(np.pi * z))

ax.quiver(x, y, z, u, v, w, length=0.1, normalize=True)

plt.show()

<IPython.core.display.Javascript object>

In [ ]:
def parse_vth(vth_file):
    vth_str = np.genfromtxt(vth_file, delimiter = ' ', dtype='unicode')
    print(vth_str.shape)
    print(vth_str[0])
    vth_dat = []
    for vth_row in vth_str:
        vth_dat.append(np.float32(vth_row.split()))
    vth_dat = np.array(vth_dat, dtype=np.float32)
    print(vth_dat.shape)
    return vth_dat


def parse_z(z_file):
    z_str = np.genfromtxt(z_file, delimiter = ' ', dtype='unicode')
    print(z_str.shape)
    print(z_str[:5])
    z_dat = []
    for z_row in z_str[5:]: # Parse from the line number 5th
        z_dat.append(np.float32(z_row.split()))
    z_dat = np.array(z_dat, dtype=np.float32)
    print(z_dat.shape)
    return z_dat

# Load the files of depth and vth
vth_dir = 'data/VTH/'
z_dir = 'data/Z/'

vth_files = natsorted(glob.glob(os.path.join(vth_dir, '*.dat')))
z_files = natsorted(glob.glob(os.path.join(z_dir, '*.grd')))

# print(vth_files[:10])
# print(z_files[:10])
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')

idx = 0
for vth_file, z_file in zip(vth_files[:], z_files[:]):
    idx = idx+1
    print('*'*50)
    
    # Read data from vth_file
    vth_dat = parse_vth(vth_file=vth_file)
    
    # Read data from z_file
    z_dat = parse_z(z_file=z_file)
    dimy, dimx = z_dat.shape
    
    # Take the magnitude
    mag = np.zeros((dimy, dimx), dtype=np.float32)
    mag[vth_dat[...,2].astype(np.int32)-1, 
        vth_dat[...,1].astype(np.int32)-1] = vth_dat[...,3] # Magnitude
    
    # Take the angle
    deg = np.zeros((dimy, dimx), dtype=np.float32)
    deg[vth_dat[...,2].astype(np.int32)-1, 
        vth_dat[...,1].astype(np.int32)-1] = vth_dat[...,4] # Phase
    
    # Convert angle to degree
    rad = np.deg2rad(deg)
    cplx = mag * np.exp(1j*rad)
    
    rx = np.real(cplx) 
    ry = np.imag(cplx) 


    norm = Normalize()
    norm.autoscale(rad.flatten())
    cmap = plt.cm.PiYG #PiYG #BuGn

    ##################################################################################
  

    yy, xx, zz = np.mgrid[0:dimy,0:dimx, 0:0.4:0.4]
    #print(zz)
    print(xx.shape, yy.shape, zz.shape)
   
    
    u = np.expand_dims(rx, -1)
    v = np.expand_dims(ry, -1)
    rz = np.expand_dims(z_dat, -1)
    w = np.zeros_like(rz)
    #print(rz.max())
    #print(rz.min())
    #print(rz)
    #print(rx.shape, ry.shape, z_dat.shape)
    
    s = 4
    i = idx%3 # For alternating the quiver
    
# Keyword arguments:
    x = xx.reshape(np.product(xx.shape))
    y = yy.reshape(np.product(yy.shape))
    z = zz.reshape(np.product(zz.shape))
    # length: [1.0 | float]
    # The length of each quiver, default to 1.0, the unit is the same with the axes
    # arrow_length_ratio: [0.3 | float]
    # The ratio of the arrow head with respect to the quiver, default to 0.3
    # pivot: [ 'tail' | 'middle' | 'tip' ]
    # The part of the arrow that is at the grid point; the arrow rotates about this point, hence the name pivot. 
    # Default is 'tail'
    # normalize: bool
    # When True, all of the arrows will be the same length. 
    # This defaults to False, where the arrows will be different lengths depending on the values of u,v,w.
    # Any additional keyword arguments are delegated to LineCollection
    plt.cla()
    
    ax.quiver3D(xx[i::s,i::s,:], yy[i::s,i::s,:], rz[i::s,i::s,:], 
              #rx[i::s,i::s,:], ry[i::s,i::s,:], 0.01*z_dat[i::s,i::s,:], 
                u[i::s,i::s,:],
                v[i::s,i::s,:],
                w[i::s,i::s,:],
                length=0.01,
                arrow_length_ratio=0.3,
                pivot='tail',
                normalize=True,
                color=cmap(norm(rad.flatten())),
              #pivot='tail', 
              #color=cmap(norm(rad.flatten())),
              #length=0.1,
              #lw=2,
              #normalize=False,
              #arrow_length_ratio=0.1,
              )
  
    # Set the axis off
    #ax.axis('equal')
    ax.axis('off')
    ax.set_xlim3d(0, dimx)
    ax.set_ylim3d(0, dimy)
    ax.set_zlim3d(-0.1, 0.3)

    
    ax.view_init(-150, -170)
    ##################################################################################
    # Plot surface
    ax.plot_surface(np.squeeze(xx), 
                    np.squeeze(yy), 
                    np.squeeze(rz), 
                    cmap=cm.PiYG,
                    rstride=8, cstride=8, alpha=0.3)
    
    cset = ax.contour(np.squeeze(xx), 
                      np.squeeze(yy), 
                      np.squeeze(rz), zdir='z', offset=0.3, cmap=cm.coolwarm)
    cset = ax.contour(np.squeeze(xx), 
                      np.squeeze(yy), 
                      np.squeeze(rz), zdir='z', offset=0.0, cmap=cm.coolwarm, 
                      extend3d=True, 
                      stride=0.05)

    
    plt.show()
    head, tail = os.path.split(vth_file)
    plt.savefig(tail.replace('.dat', '.png'), bbox_inches='tight')
# 

<IPython.core.display.Javascript object>

**************************************************
(63684,)
360	2	2	0	0
(63684, 5)
(250,)
['DSAA' '262\t245' '1\t262' '1\t245' '-5\t5']
(245, 262)
(245, 262, 1) (245, 262, 1) (245, 262, 1)
**************************************************
(63684,)
720	2	2	0	0
(63684, 5)
(250,)
['DSAA' '262\t245' '1\t262' '1\t245' '-5\t5']
(245, 262)
(245, 262, 1) (245, 262, 1) (245, 262, 1)
**************************************************
(63684,)
1080	2	2	0	0
(63684, 5)
(250,)
['DSAA' '262\t245' '1\t262' '1\t245' '-5\t5']
(245, 262)
(245, 262, 1) (245, 262, 1) (245, 262, 1)
**************************************************
(63684,)
1440	2	2	0	0
(63684, 5)
(250,)
['DSAA' '262\t245' '1\t262' '1\t245' '-5\t5']
(245, 262)
(245, 262, 1) (245, 262, 1) (245, 262, 1)
